In [1]:
import os


import pprint
import random
import numpy as np
import pandas as pd

import spacy
import sys
sys.path.append("../")
from datatools.analyzer import *
# from datatools.analyzer import clean_text
from error_tools import *

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
from nltk.lm        import Vocabulary
from nltk.lm.models import MLE
from nltk.util      import ngrams

In [3]:
class Utterance:
    utt_level = ["Wrong information", "Semantic error", "Uninterpretable", "Grammatical error"]
    def __init__(self, did, sp, utt, errors, type_) -> None:
        self.did = did
        self.sp = sp
        self.utt = utt
        self.errors = errors
        self.type_ = type_
    
    def __str__(self):
        return "{0}: {1}".format(self.sp, self.utt)

    def is_system(self):
        return True if self.sp=="S" else False
    
    def is_error_included(self, error):
        # Null 対応
        if not self.errors:
            return False
        return error in self.errors
    
    def is_exist_error(self):
        return True if self.errors else False
    
    def is_type_included(self, type_):
        return type_ in self.type_
    
    def is_utt_level_error(self):
        for e in Utterance.utt_level:
            if self.is_error_included(e):
                return True
        return False
    

import json
from pathlib import Path
def read_conv(path:str, datalist:list):
    convs = []
    for p in datalist:
        datapath = Path(path + p + '/')
        for file_ in datapath.glob("*.json"):
            conv = []
            with open(file_, "r") as f:
                json_data = json.load(f)
                did = json_data["did"]
                for t in json_data["turns"]:
                    sp = t["speaker"]
                    utt = t["utterance"]
                    errors = t["error_category"]
                    type_ = t["type"]
                    one = Utterance(did, sp, utt, errors, type_)
                    conv.append(one)
            convs.append(conv)
    return convs  

In [4]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
error_types = ['Unclear intention', 'Wrong information',
 'Ignore question', 'Topic transition error', 
 'Lack of information', 'Repetition', 
 'Contradiction', 'Self-contradiction',
  'Lack of common sense', 'Semantic error',
   'Grammatical error', 'Ignore proposal', 
   'Ignore offer', 'Lack of sociality', 
   'Uninterpretable', 'Ignore greeting', 
   'No-Err']


In [5]:
convs = read_conv(path, datalist)

In [6]:
usr_utt = []
for conv in convs:
    for i, ut in enumerate(conv):
        if not ut.is_system():
            usr_utt.append(clean_text(ut.utt))
            

In [8]:
usr_utt[:15]

['そうですね。最近とても暑いですから。',
 '休日に行きたいと思います。',
 'はい。あなたは海に行きますか？',
 '何故ですか？',
 'そうですか。山に行くのはどうでしょうか？',
 '山はお好きなのですか？',
 '山の御勧めのスポットは御存知ですか？',
 'どこに行くといいですか？',
 '明日はとても暑くなるみたいですね。',
 '涼しくなってきたら、一緒に山へ行きたいですね。',
 '美味しいですね。',
 'でも高いですね',
 '1000円ぐらいかな？',
 '意味が分かりません',
 'サスペンスが好きです']

In [10]:
filled_normal = fill_SYMBOL( sentence2normalize_nv(usr_utt) )

In [18]:
def create_language_model(sentences, N):
    vocab = Vocabulary([word for sent in sentences for word in sent])
    text_ngrams = [ngrams(sent, N) for sent in sentences]
    lm = MLE(order=N, vocabulary=vocab)
    lm.fit(text_ngrams)
    return lm

In [19]:
lm = create_language_model(filled_normal, N=3)

In [21]:
context = ("そう", 'です')

In [24]:
usr_utt[:15]

['そうですね。最近とても暑いですから。',
 '休日に行きたいと思います。',
 'はい。あなたは海に行きますか？',
 '何故ですか？',
 'そうですか。山に行くのはどうでしょうか？',
 '山はお好きなのですか？',
 '山の御勧めのスポットは御存知ですか？',
 'どこに行くといいですか？',
 '明日はとても暑くなるみたいですね。',
 '涼しくなってきたら、一緒に山へ行きたいですね。',
 '美味しいですね。',
 'でも高いですね',
 '1000円ぐらいかな？',
 '意味が分かりません',
 'サスペンスが好きです']

In [ ]:
print(context, '->')

In [22]:
prob_list = []
for word in lm.context_counts(lm.vocab.lookup(context)): # 文脈に続く単語一覧の取得
    prob_list.append((word, lm.score(word, context))) # 単語のその出現する確率を格納

prob_list.sort(key=lambda x: x[1], reverse=True) # 出現確率順にソート
for word, prob in prob_list:
    print('\t{:s}: {:f}'.format(word, prob))

	ね: 0.734513
	か: 0.150442
	よ: 0.070796
	。: 0.017699
	けど: 0.008850
	EOS: 0.008850
	が: 0.008850


In [46]:
import math
def sentence2score(sentence, l, N):
    filled = fill_SYMBOL( sentence2normalize_nv(sentence) )
    ngram_text = []
    for L in filled:
        for i in range(len(L)-N+1):
            # print(L[i:i+N])
            ngram_text.append(L[i:i+N])
    
    all_score = 0
    for ngram in ngram_text:
        context = (ngram[:-1])
        # print(context)
        # for word in lm.context_counts(lm.vocab.lookup(context)): # 文脈に続く単語一覧の取得
            
        score = lm.score(ngram[-1], context) + 1e-4
        print(score)
        log_score = math.log2(score)
        all_score += log_score
    print(all_score/len(ngram_text))

    

In [47]:
sentence = "そうですね。最近とても暑いですから。"
sentence2score(sentence, lm, N=3)

0.4251
0.7346132743362832
0.6097256684491978
0.9932972789115646
0.005335602094240838
1.0001
1.0001
0.13343333333333332
0.2223222222222222
1.0001
-1.502732464507734


In [51]:
sentence = "もちろんですってのが元気ですかにいくないですよ"
sentence2score(sentence, lm, N=3)

0.3334333333333333
0.0001
0.0001
0.0001
0.0001
0.3334333333333333
0.2778777777777778
0.0001
0.0001
0.007974015748031495
0.1876
0.21749130434782607
0.1781821917808219
-7.601315602452995


In [54]:
sentence = "パスタが好きですね"
sentence2score(sentence, lm, N=3)

0.1070364161849711
0.2246508982035928
0.6870565217391305
0.056350000000000004
0.28619625668449195
-2.374786563949037
